In the last notebook, we worked with chunking since there is a token limit with large language models. If we have lots and lots of data, that means we will have lots of chunks created as well. Is there a way to select the most relevant chunks of text? The answer is yes. To solve this problem, we can take a look at a process called Embedding. Embedding helps us create numerical representations for all the chunks. Then, we can find the most similar chunks in the the list of embeddings. One popular way to find the similar chunks is through cosine similarity.

One thing to keep in mind is that embeddings can be created for all different data types- images, audio, video, and text. 


Notes---
- embeddings: what are they? why are they imp? and esp now?
- cosine similarity 
- think about diff types of data and how everything can be converted into embeddings 
- help with finding similarity 
- talk about the different applications 

CH4 (txt similarity)
CH3 images, fraud transactions, personalization

note: take a look at redis for documentation

In [8]:
import openai
import os
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt
load_dotenv() 
openai.api_type = "azure"
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_version = "2023-05-15"
embedding_model=os.getenv("EMBEDDING_MODEL_NAME")


In [26]:
embedding = openai.Embedding.create(
    input="hi my name is devanshi", engine=embedding_model
)["data"][0]["embedding"]
print(embedding)
len(embedding)

c:\Users\dthakar\OneDrive\Documents\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[-0.02204902283847332, -0.0019086977699771523, -0.012975976802408695, -0.022898036986589432, -0.013926365412771702, 0.05182787403464317, -0.028030136600136757, -0.002472595078870654, -0.0075714318081736565, -0.007970594801008701, 0.018817700445652008, 0.0006997238378971815, 0.007165932562202215, -0.0009258372010663152, -0.01284292247146368, -0.015611722134053707, 0.025381719693541527, -0.013216741383075714, 0.015282253734767437, -0.020895883440971375, -0.030032290145754814, 0.012317040003836155, 0.021085962653160095, 0.0011127470061182976, -0.032617345452308655, 0.0014049916062504053, 0.02899319678544998, -0.013229413889348507, 0.005610462743788958, -0.009453201666474342, -0.01959068328142166, 0.013001319952309132, 0.015244238078594208, -0.030032290145754814, 0.01129062008112669, -0.01893174648284912, 0.0047075930051505566, -0.00828105490654707, 0.013178725726902485, -0.0033073534723371267, 0.02291070856153965, -0.012621164321899414, -0.01581447198987007, -0.007191276177763939, -0.0179

1536

In [16]:
%pip install mwclient
%pip install mwparserfromhell

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [17]:
# imports
import mwclient  # for downloading example Wikipedia articles
import mwparserfromhell  # for splitting Wikipedia articles into sections
import openai  # for generating embeddings
import pandas as pd  # for DataFrames to store article sections and embeddings
import re  # for cutting <ref> links out of Wikipedia articles
import tiktoken  # for counting tokens

In [19]:
# get Wikipedia pages about the 2022 Winter Olympics

CATEGORY_TITLE = "Category:2022 Winter Olympics"
WIKI_SITE = "en.wikipedia.org"


def titles_from_category(
    category: mwclient.listing.Category, max_depth: int
) :
    """Return a set of page titles in a given Wiki category and its subcategories."""
    titles = set()
    for cm in category.members():
        if type(cm) == mwclient.page.Page:
            # ^type() used instead of isinstance() to catch match w/ no inheritance
            titles.add(cm.name)
        elif isinstance(cm, mwclient.listing.Category) and max_depth > 0:
            deeper_titles = titles_from_category(cm, max_depth=max_depth - 1)
            titles.update(deeper_titles)
    return titles


site = mwclient.Site(WIKI_SITE)
category_page = site.pages[CATEGORY_TITLE]
titles = titles_from_category(category_page, max_depth=1)
# ^note: max_depth=1 means we go one level deep in the category tree
print(f"Found {len(titles)} article titles in {CATEGORY_TITLE}.")

c:\Users\dthakar\OneDrive\Documents\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Found 732 article titles in Category:2022 Winter Olympics.


In [24]:
# define functions to split Wikipedia pages into sections

SECTIONS_TO_IGNORE = [
    "See also",
    "References",
    "External links",
    "Further reading",
    "Footnotes",
    "Bibliography",
    "Sources",
    "Citations",
    "Literature",
    "Footnotes",
    "Notes and references",
    "Photo gallery",
    "Works cited",
    "Photos",
    "Gallery",
    "Notes",
    "References and sources",
    "References and notes",
]


def all_subsections_from_section(
    section: mwparserfromhell.wikicode.Wikicode,
    parent_titles: list,
    sections_to_ignore: set,
) :
    """
    From a Wikipedia section, return a flattened list of all nested subsections.
    Each subsection is a tuple, where:
        - the first element is a list of parent subtitles, starting with the page title
        - the second element is the text of the subsection (but not any children)
    """
    headings = [str(h) for h in section.filter_headings()]
    title = headings[0]
    if title.strip("=" + " ") in sections_to_ignore:
        # ^wiki headings are wrapped like "== Heading =="
        return []
    titles = parent_titles + [title]
    full_text = str(section)
    section_text = full_text.split(title)[1]
    if len(headings) == 1:
        return [(titles, section_text)]
    else:
        first_subtitle = headings[1]
        section_text = section_text.split(first_subtitle)[0]
        results = [(titles, section_text)]
        for subsection in section.get_sections(levels=[len(titles) + 1]):
            results.extend(all_subsections_from_section(subsection, titles, sections_to_ignore))
        return results


def all_subsections_from_title(
    title: str,
    sections_to_ignore: set = SECTIONS_TO_IGNORE,
    site_name: str = WIKI_SITE,
) :
    """From a Wikipedia page title, return a flattened list of all nested subsections.
    Each subsection is a tuple, where:
        - the first element is a list of parent subtitles, starting with the page title
        - the second element is the text of the subsection (but not any children)
    """
    site = mwclient.Site(site_name)
    page = site.pages[title]
    text = page.text()
    parsed_text = mwparserfromhell.parse(text)
    headings = [str(h) for h in parsed_text.filter_headings()]
    if headings:
        summary_text = str(parsed_text).split(headings[0])[0]
    else:
        summary_text = str(parsed_text)
    results = [([title], summary_text)]
    for subsection in parsed_text.get_sections(levels=[2]):
        results.extend(all_subsections_from_section(subsection, [title], sections_to_ignore))
    return results

c:\Users\dthakar\OneDrive\Documents\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
# split pages into sections
# may take ~1 minute per 100 articles
wikipedia_sections = []
for title in titles:
    wikipedia_sections.extend(all_subsections_from_title(title))
print(f"Found {len(wikipedia_sections)} sections in {len(titles)} pages.")

c:\Users\dthakar\OneDrive\Documents\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Found 5728 sections in 732 pages.


In [ ]:
# clean text
def clean_section(section: list) :
    """
    Return a cleaned up section with:
        - <ref>xyz</ref> patterns removed
        - leading/trailing whitespace removed
    """
    titles, text = section
    text = re.sub(r"<ref.*?</ref>", "", text)
    text = text.strip()
    return (titles, text)


wikipedia_sections = [clean_section(ws) for ws in wikipedia_sections]

# filter out short/blank sections
def keep_section(section: list) -> bool:
    """Return True if the section should be kept, False otherwise."""
    titles, text = section
    if len(text) < 16:
        return False
    else:
        return True


original_num_sections = len(wikipedia_sections)
wikipedia_sections = [ws for ws in wikipedia_sections if keep_section(ws)]
print(f"Filtered out {original_num_sections-len(wikipedia_sections)} sections, leaving {len(wikipedia_sections)} sections.")